In [1]:
# using the CM metadata, create an SOLR index with pages' metadata
import os, pprint, sys, time, csv, json
root = '/data/cm/data/'
root_out = '/data/cm/output/'

In [2]:
metadata = list(csv.reader(open(root+'metadata/CM_Seiten_Metadaten.csv', 'r'), delimiter='\t'))

In [3]:
j_metadata = json.load(open('/data/scripts/cm/scripts/metadata/j_metadata.json', 'r', encoding='utf-8'))

In [4]:
for k,v in j_metadata.items():
    print(k,v)
    break

2710055 {'dateIssued': '1919', 'place': 'München', 'ident': '2296512-9', 'lang': 'ger', 'subtitle': 'Zeitschr. der Jugend', 'title': 'Der neue Anfang', 'publisher': 'Verl. Der Neue Anfang'}


In [5]:
print(metadata [0])
metadata  = {line[0]: line for line in metadata [1:]}

['VLID_Seite', 'OT_PATH', 'VLID_Zs', 'VLID_Parent', 'Parent Knotentyp', 'Parent-Type', 'Zs_Caption', 'Volume_Caption', 'Heft_Caption', 'Aufsatz_Caption', 'Datum', 'Seite (OT_SORT)', 'Seite_Caption', 'Seitenzahl_Caption']


In [6]:
# check overlap between VLIDs in my text repository and VLIDs in metadata

overlap = {'is_there': 0, 'is_not_there': 0}
for r, dirs, files in os.walk(root_out+'text/'):
    for file in files:
        try:
            vlid = file.split('.')[0].split('_')[1]
            if vlid in metadata:
                overlap['is_there'] +=1
            else:
                overlap['is_not_there'] += 1
        except IndexError:
            print(r, file)

/data/cm/output/text/ README.md
/data/cm/output/text/ .gitattributes
/data/cm/output/text/ wissenschaft.zip
/data/cm/output/text/ texts-complete-2022-09-12.tar.gz
/data/cm/output/text/ orthodoxie.zip
/data/cm/output/text/ fancyindex.conf
/data/cm/output/text/Nginx-Fancyindex-Theme-light addNginxFancyIndexForm.js
/data/cm/output/text/Nginx-Fancyindex-Theme-light header.html
/data/cm/output/text/Nginx-Fancyindex-Theme-light styles.css
/data/cm/output/text/Nginx-Fancyindex-Theme-light footer.html
/data/cm/output/text/Nginx-Fancyindex-Theme-light showdown.min.js
/data/cm/output/text/Nginx-Fancyindex-Theme-light jquery.min.js


In [7]:
for journal in os.listdir(root_out+'text/'):
    
    start = time.time()
    print('Processing', journal+'...', end=" ")
    
    # create a new folder for output (don't create the whole hierarchy, only journal folder)
    if not os.path.exists(root_out+"solr_meta_fra/"+journal):
        os.makedirs(root_out+"solr_meta_fra/"+journal)
        
    for r, dirs, files in os.walk(root_out+'text/'+journal+'/'):
        
        for file in files:
            filename = file.split('.')[0]
            vlid = file.split('.')[0].split('_')[1]
            
            if vlid in metadata:
            
                text = open(r+'/'+file).read()
            
                try:
                    j_title = metadata[vlid][6]
                except Error:
                    j_title = ""
                    
                try:
                    lang = j_metadata[journal]['lang']
                except KeyError:
                    lang = ""
                    
                try:
                    publisher = j_metadata[journal]['publisher']
                except KeyError:
                    publisher = ""
                    
                try:
                    volume = metadata[vlid][7]
                except Error:
                    volume = ""
                
                try:
                    heft = metadata[vlid][8]
                except Error:
                    heft = ""
                    
                try:
                    aufsatz = metadata[vlid][9]
                except Error:
                    aufsatz = ""
                    
                try:
                    date = metadata[vlid][10]
                except Error:
                    date = ""
                    
                try:
                    place = j_metadata[journal]['place']
                except KeyError:
                    place = ""
                    
                try:
                    if metadata[vlid][13] != "(null)":
                        page = metadata[vlid][13]
                    else:
                        page = ""
                except Error:
                    page = ""
                    
                try:
                    vlid_journal = metadata[vlid][2]
                except Error:
                    vlid_journal = ""
                    
                try:
                    vlid_page = metadata[vlid][0]
                except Error:
                    vlid_page = ""

                solr = []
            
                # Solr document body 
                solr_doc = {
                    "id": filename,
                    "text": text,
                    "dateIssued": date,
                    "lang": lang,       
                    "place": place,
                    "j_title": j_title,
                    "publisher": publisher,
                    "volume": volume,
                    "heft": heft,
                    "aufsatz": aufsatz,
                    "page": page,
                    "vlid_journal": vlid_journal,
                    "vlid_page": vlid_page,
                }

                # JSONL line for Solr
                line = json.dumps(solr_doc, ensure_ascii=False)

                # Append to output list
                solr.append(line)
                with open(root_out+"solr_meta_fra/"+journal+"/"+filename+".json",
                          "w", encoding="utf-8") as write_file:
                    write_file.write("\n".join(solr))

                    
    end = time.time()
    task_duration = round(end-start, 2)
    print('done in', str(task_duration))

Processing 2710055... done in 0.05
Processing 9616701... done in 0.94
Processing 2580773... done in 1.09
Processing 2895450... done in 3.66
Processing 2727810... done in 2.15
Processing 3129675... done in 1.39
Processing 6492429... done in 0.43
Processing 2438141... done in 4.24
Processing 2827798... done in 0.33
Processing 5438908... done in 0.09
Processing 4782723... done in 1.42
Processing 7506414... done in 0.34
Processing 9054153... done in 0.02
Processing 9366335... done in 0.01
Processing 4889892... done in 0.21
Processing 2425321... done in 0.5
Processing 6492329... done in 0.07
Processing 700673... done in 0.01
Processing 4895413... done in 0.21
Processing 2542917... done in 2.02
Processing 9583346... done in 0.12
Processing 2549643... done in 0.12
Processing 10638461... done in 0.2
Processing 3139318... done in 0.47
Processing 9620169... done in 0.16
Processing 9051331... done in 0.0
Processing 4781937... done in 0.1
Processing 4911569... done in 0.02
Processing 2625941... do

IndexError: list index out of range

In [ ]:
langs = {}
for k,v in j_metadata.items():
    if v['lang'] not in langs.keys():
        langs[v['lang']] = 0
    langs[v['lang']] += 1
print(langs)